# Binding Affinity Prediction with ML-Based Docking

Let's download the data and create the environment while we talk about some of the theory and the workshop structure.

In [ ]:
import os
os.makedirs("data", exist_ok=True)
os.makedirs("figures", exist_ok=True)
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/data/6vhn.pdb" -O "data/6vhn.pdb"
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/data/6vhn_prepared.pdb" -O "data/6vhn_prepared.pdb"
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/data/Enamine_Hinge_Binders_Library_plated_24000cmds_20210316%20(1).sdf" -O "data/Enamine_Hinge_Binders_Library_plated_24000cmds_20210316%20(1).sdf"
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/__init__.py" "__init__.py"
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/workshop_2_utils.py" "workshop_2_utils.py"
!wget "https://raw.githubusercontent.com/valence-labs/mtl_summer_school_2024/main/Lab2/env.yml" "env.yml"
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # kernel will die and restart. This is expected.

In [ ]:
import condacolab
condacolab.check()

In [ ]:
# Updating the base environment, this will take a bit (~2min)
!mamba env update -n base -f env.yml

In [ ]:
!pip install py3Dmol #For rendering

In [ ]:
from google.colab import output
output.enable_custom_widget_manager() # for output rendering



## PREMISE

In this lab, we will use molecular docking to predict the binding pose and affinity of small molecules in a protein binding site.

For the remainder of the labs, we will use the **epidermal growth factor receptor \(EGFR\)** as our target system \([6VHN](https://www.rcsb.org/structure/6VHN)\).
The protein encoded by this gene is a transmembrane glycoprotein that is a member of the protein kinase superfamily.
We are going to predict the **binding affinity** of some molecules to this protein to the ATP binding site in the hinge region.

EGFR is a frequently over-expressed and aberrantly activated trans-membrane protein in non-small cell lung cancer (NSCLC) patients, described for the first time in 2004. Mutations in this gene are associated with lung cancer in particular.

The goal of this lab is twofold:
- First, we explain important steps of a **molecular docking workflow** using the classical docking software Autodock Vina.
- Second, we build an **active learning / Bayesian optimization** cycle to find high binding affinity candidates from a molecular screening library while minimizing the number of evaluations of the \(somewhat expensive\) Docking oracle.

Such a pipelines could be used in the hit\-to\-lead phase of the drug\-discovery process or to reduce the size of the screening library to be tested.

Such active learning pipelines can be also used in other phases of the drug\-discovery process such as in lead-optimization or to select the best candidates for experimental/wet lab validation.

The docking part of this lab is inspired by the following TeachOpenCADD tutorial: [https://projects.volkamerlab.org/teachopencadd/talktorials/T015\_protein\_ligand\_docking.html](https://projects.volkamerlab.org/teachopencadd/talktorials/T015_protein_ligand_docking.html)

<img src="http://drive.google.com/uc?export=view&id=1Ho4AmpFwznTliRiWBGV7Bp7CxZl64EiA" alt="drawing" width="800"/>

**Fig. 1:** Rendering (VMD) of EGFR (6VHN) in complex with inhibitor.

### Molecular docking

In the modern drug discovery pipeline, determining the binding mode and binding affinity of an active molecule to a given protein target is very important. With the advent of **on\-demand synthesis libraries** such as [Enamine Real](https://enamine.net/compound-collections/real-compounds/real-database), the design space of possible ligands extends to billions of theoretically synthesizable molecules. Molecular docking can probe and \(de\)prioritize these molecules before they are even synthesized, thus, accelerate the discovery of novel lead candidates.

Molecular docking software can be used to **predict binding modes and affinities** by sampling possible conformations of a ligand inside the protein binding pocket \(Fig. 2\). To this end, the sampling of conformations aims to optimize a Docking score function. After the optimal pose has been found, which is typically calculated from a variety of terms for different non\-covalent molecular interactions, the Docking score is used to predict the binding free energy.

<img src="http://drive.google.com/uc?export=view&id=1EiXraYCjASFXMiiOsQbV4V_cCLJzbWW9" alt="drawing" width="800"/>

**Fig. 2:** EGFR in complex with inhibitor \(_David Schaller_\).



### Background: Binding free energy

We are interested in finding drug compounds that strongly bind to a target protein.
Hence, in the equilibrium reaction between protein $P$ and ligand $L$
\begin{equation}
 P + L 	\rightleftharpoons PL\ ,
\end{equation}
we are interested in a large ratio of protein-ligand complexes over proteins and ligands individually.


<img src="http://drive.google.com/uc?export=view&id=1du-Nt5j3CwrIKU3AIK8TA3Jv1jfkdco1" alt="drawing" width="400"/>

**Fig. 3:** Protein-ligand binding equilibrium reaction \([Amber Tutorial](https://ambermd.org/tutorials/advanced/tutorial3/), _Ross Walker_\, 2006).

This ratio is proportional to the binding constant $K_b^0$, which, on the other hand, is proportional to the **Gibbs free energy of binding**
\begin{equation}
 \Delta G^0_\mathrm{bind} = -k_B T \ln K_b^0 \ ,
\end{equation}
where $k_B$ is the Boltzmann constant and $T$ is the temperature. Hence, we are interested in predicting $ \Delta G^0_\mathrm{bind}$ in-silico. While $ \Delta G^0_\mathrm{bind}$ can be computed physically rigorous via MD simulations, this is too computationally expensive for large-scale screenings in the hit-to-lead phase. To this end, we can use Docking to provide us with a less accurate, but significantly less computationally expensive estimate of $ \Delta G^0_\mathrm{bind}$ in order to (de-)prioritize compounds.



### Docking score

**TLDR** : An empirical mathematical function used to __proxy__ the binding free energy of a protein-ligand complex. Used to rank ligands in a virtual screening by considering
the inter- & intra molecular interactions and other physical properties (H-bonds, hydrophobic interactions, etc.) to define a scalar value.

The docking score c in [AutoDock Vina](https://onlinelibrary.wiley.com/doi/10.1002/jcc.21334) is built of pairwise physics\-inspired functions f
\begin{equation}
c = \sum_{i<j} f_{t_it_j}(r_{ij}) \ ,
\end{equation}
where $t_i$ is the atom type and $r_{ij}$ is the pairwise distance between atoms $i$ and $j$.
The Docking score consists of both intra and inter\-molecular interactions
\begin{equation}
c =c_\mathrm{inter} + c_\mathrm{intra} \ ,
\end{equation}
but only the inter\-molecular component is used to predict the binding free energy:
\begin{equation}
 \Delta G^0_\mathrm{bind} \approx g(c_\mathrm{inter}) = \frac{c_\mathrm{inter}}{1+w N_\mathrm{rot}} \ ,
\end{equation}
where $N_\mathrm{rot}$ is the number of rotable bonds of the ligand and $w$ is a weight parameter.
The parameters of $c$ are optimized based on the PDBBind database.



### Sampling algorithms

To sample the conformations of ligands inside a protein binding pocket \([Curr Comput Aided Drug Des \(2011\), 7, 2, 146\-157](https://doi.org/10.2174/157340911795677602)\), classical Docking algorithms employ sampling algorithms such as the following:

* **Matching algorithms:** Compare the shape similarity of ligand conformations and the protein binding pocket, usually also including chemical information, e.g. hydrogen bond acceptors and donors. However, these programs require a _prior computation of ligand conformations_ that are used during shape comparison.This will fail if the biologically relevant conformation is not present in this library.

* In the **incremental instruction** method, the ligand is first deconstructed into smaller fragments by breaking its rotatable bonds. One of the fragments, for example the biggest one, is placed first into the binding pocket. Subsequently, the complete ligand is _incrementally constructed inside the binding pocket_ by connecting the remaining fragments at the appropriate positions of the core fragment.

* **Monte Carlo methods** sample ligand conformations by rigid\-body rotation and translation as well as bond rotation. They generate random placements and evaluate obtained conformations inside the protein binding pocket with the pose score function. If the pose is accepted, the conformation is saved and subsequently randomly modified to generate another conformations.

* **Genetic algorithms** encode information about the ligand configuration in so-called _chromosomes_. Then, genetic operations such as mutation are performed. Finally, the population of _chromosomes_ is evaluated by the docking score and preferential selection of high-performing _chromosomes_ for genetic cross-over result in an updated, (hopefully) more optimal population.

Vina uses a genetic algorithm for global search and optimizes local minima using the BFGS quasi-Newton optimizer.



A **molecular docking** workflow usually involves the following steps \(Fig. 4\):

- Input file preparation, e.g. protonation and conversion into specific file formats
- Conformational sampling of the ligand inside the binding pocket via classical optimization of the docking score
- Scoring of the generated docking poses via and docking score $c$ and prediction of $\Delta G^0_\mathrm{bind}$ via $g(c_\mathrm{inter})$
- Post\-processing, e.g. visualization of the docking pose

<img src="http://drive.google.com/uc?export=view&id=1SSd-cX7-5hKdUSxIhZnEJQ_rAVbjHr6M" alt="drawing" width="800"/>

__Fig. 4:__ Molecular docking workflow, (_Michele Wichmann_ and _David Schaller_)



## Implementation

The implementation consists of 2 parts: First, we will prepare the Protein and compound database for Docking and _deomonstrate Docking of a single compound_. In the second part, we will use this pipeline to dock batches of compounds and _actively learn a Gaussian Process_ surrogate model of the docking score for efficient screening.

### Input preparation

In this workshop we will consider the simplest case of rigid docking with a known binding site. The information of the binding site will be provided by the crystallized ligand in the PDB entry.

In a real case scenario, the binding site can be unknown. In this case, the research of the binding site can be done through various techniques or be achieved through blind docking. Recently, ML methods \(e.g. DiffDock\) have shown great potential in this task.



In [ ]:
import mdtraj
import numpy as np
import os
from workshop_2_utils import *


os.makedirs("sdf_inputs", exist_ok=True)
os.makedirs("smina_inputs", exist_ok=True)



In [ ]:
traj = mdtraj.load("data/6vhn.pdb")

def get_protein_ligand_idxs(traj ,resname=None):
    protein = traj.top.select("protein")
    resname = "not protein" if not resname else resname
    ligand = traj.top.select(resname)
    return protein, ligand

def save_trimmed_pdb(path, traj, idxs):
    traj.atom_slice(idxs).save_pdb(path)


receptor, ligand = get_protein_ligand_idxs(traj, "not protein and not water")

save_trimmed_pdb("data/ligand.pdb", traj,ligand)
#save_trimmed_pdb("data/receptor.pdb", traj,receptor)



### Preparing a pdb

For the docking, we need to prepare a pdb file of the protein and the ligand.
For this workshop, the protein pdb was already prepared  by performing the following steps:

- Removed the ligand from the pdb file
- Deleted all the water molecules/solvent from the pdb file
- Converted residues to standard residues  
- Completed sidechains
- Added hydrogens to the protein to the correct protonation state (ph 7.4)
- Added charges to the protein (Gasteiger model)
- Changed names of the residues to AMBER ff14Sb names

Multiple programs can be used to complete this steps, e.g. med-chem programs including Maestro, Chimera, etc. or python libraries such as openmm and pbdfixer

In [ ]:
import py3Dmol
# First we assign the py3Dmol.view as view
view=py3Dmol.view()
# The following lines are used to add the addModel class
# to read the PDB files of chain B and C
view.addModel(open('data/6vhn_prepared.pdb', 'r').read(),'pdb')
view.addModel(open('data/6vhn.pdb', 'r').read(),'pdb')
# Zooming into all visualized structures
view.zoomTo()
# Here we set the background color as white and set the cartoon style
view.setBackgroundColor('white')
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
# And we finally visualize the structures using the command below
view.show()

In [ ]:
def read_pdb_with_ob(file):
    """Read a molecule file with open babel

    Args:
        file (Union[str os.PathLike]): pdb input file

    Returns:
        mols (list): list of molecules found in the input file
    """

    try:
        from openbabel import pybel
    except ImportError:
        raise ImportError("Pybel is required for reading openbabel molecules")
    mols = [m for m in pybel.readfile(format="pdb",filename=file)]
    return mols

def prepare_ob_mols(ligand, outpath, overwrite=False):
    from openbabel import pybel
    out = pybel.Outputfile(format="pdbqt" , filename=outpath,  overwrite=overwrite)
    ligand.addh()
    if not ligand.OBMol.HasNonZeroCoords():
        ligand.make3D()
    ligand.calccharges(model="gasteiger")
    out.write(ligand)
    out.close()

ligand_mol= read_pdb_with_ob("data/ligand.pdb")


In [ ]:
ligand_mol[0]

In [ ]:
prepare_ob_mols(ligand_mol[0], "smina_inputs/ligand.pdbqt", overwrite=True)

In [ ]:
prep=Preprocessor()
prep.prepare_receptor("data/6vhn_prepared.pdb", "smina_inputs/receptor.pdbqt")
#prep.prepare_ligand("data/ligand.pdb", "smina_inputs/ligand.pdbqt", in_format="pdb")

## Binding box creation

- _What is a binding box?_

It is a three-dimensional area where the docking of small molecules into the protein is performed.
The box defines the region of interest on the protein where we believe the small molecule ligand might interact.

- _How do I know the binding box?_

From the crystal structure most likely! In this case we are using the binded ligand to get the position of the binding box.

- _What if I don't know the binding box?_

Then we are doing blind docking and research about the docking site is needed. We can also utilize multiple binding boxes where we believe the punitive sites are.
Methods to search for pockets like fpocket, convex hull or MD with fragments can be useful. Recent ML approaches such as [EquiBind](https://arxiv.org/abs/2202.05146) also aim to predict binding sites.


In [ ]:
ligand=mdtraj.load("data/ligand.pdb")
def create_box_from_ligand(ligand):
    xyz=ligand.xyz[0] * 10  # convert to Angstrom from nm
    pocket_center = (xyz.max(axis=0) + xyz.min(axis=0)) / 2
    pocket_size = xyz.max(axis=0) - xyz.min(axis=0) + 5
    return Box.from_array(pocket_center, pocket_size)

box=create_box_from_ligand(ligand)
box

In [ ]:
from workshop_2_utils import Docking

docker=Docking("smina_inputs/receptor.pdbqt", box)


In [ ]:
os.makedirs("outputs", exist_ok=True)
text=docker.dock_one("smina_inputs/ligand.pdbqt", "outputs/ligand_out.sdf")
docker.parse_output(text)

In [ ]:
view = py3Dmol.view()
view.addModel(open('data/6vhn_prepared.pdb', 'r').read())
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.addModel(open('outputs/ligand_out.sdf', 'r').read())
view.setStyle({'model': -1}, {"stick" :  {'color': "yellow"}})
view.zoomTo()
view.show()

In [ ]:
poses=dm.read_sdf("outputs/ligand_out.sdf", as_df=True, mol_column="mols", n_jobs=-1)
poses

In [ ]:
dm.viz.to_image(poses["mols"])

### Let's try our hand with some real molecules

In [ ]:
import datamol as dm
df_mols = dm.read_sdf("data/Enamine_Hinge_Binders_Library_plated_24000cmds_20210316%20(1).sdf", as_df=True, mol_column="mols", n_jobs=-1)
docker.parse_mol_to_pbdqt(df_mols["mols"][0]) # write mol as pdbqt



In [ ]:
text=docker.dock_one("smina_inputs/mol_0.pdbqt", "outputs/poses_0.sdf")
docker.parse_output(text)
poses=dm.read_sdf("outputs/poses_0.sdf", as_df=True, mol_column="mols", n_jobs=-1)
poses

In [ ]:
dm.viz.to_image(poses["mols"])

In [ ]:
df_mols.head()

In [ ]:
df_mols["fp"]=df_mols["mols"].apply(lambda x : dm.to_fp(x))
df_mols

In [ ]:
docker=Docking("smina_inputs/receptor.pdbqt", box)

In [ ]:
docker.dock_multiple_mols(
        df_mols["mols"].tolist()[:5], list(range(5))
)


In [ ]:
poses = dm.read_sdf("smina_outputs/poses.sdf", as_df=True, mol_column="mols", n_jobs=-1, sanitize=False)
poses.sort_values("minimizedAffinity",inplace=True)
poses

In [ ]:
dm.viz.to_image(poses["mols"].tolist()[:10])

In [ ]:
from ipywidgets import interact, Dropdown

def view_mol(molecule):
  view = py3Dmol.view(
      data=Chem.MolToMolBlock(molecule),
      #style={"sphere": {"scale" : 0.3}}
  )
  view.setStyle({"stick" : {}})
  view.addStyle({"sphere": {"scale" : 0.21}})
  view.zoomTo()
  return view.show()

mols=poses["mols"].tolist()
affs=poses["minimizedAffinity"].tolist()
smiles=poses["smiles"].tolist()

dropdown = Dropdown(
    options=[(f"{smile}:{aff} kcal/mol",mol) for aff,smile,mol in zip(affs,smiles,mols)],
    value = mols[0], description="Selection"
)
interact(
    view_mol,
    molecule=dropdown
)

In [ ]:

def create_py3d_model(sdf_file):
  molecules=dm.read_sdf(sdf_file, remove_hs=False)
  view = py3Dmol.view()
  view.addModel(open('data/6vhn_prepared.pdb', 'r').read())
  view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
  for mol in molecules:
    view.addModel(Chem.MolToMolBlock(mol,confId=0), "sdf")
    view.setStyle({'model': -1}, {"stick" :  {}})
  view.zoomTo()
  return view

view=create_py3d_model("smina_outputs/poses.sdf")
view.show()



## Active Learning

In Lab1, we have seen how to train ML models in a supervised manner. To do this we need:

- Labeled data
- A model

In the drug-discovery pipeline, data and their labels tend to be scarce. Hence, structure-based drug discovery is often conducted in low-data regime as **generating new data is expensive** and time consuming.

To tacke this problem, **active learning** is often used to choose the next samples to expensively annotate to learn better models.

In this paradigm, the model actively selects the data that it will learn from. Instead of feeding it a predefined set of training data, the model has the ability to choose the most informative samples for its training, resulting in a more efficient and effective learning process.

An **active learning** workflow usually involves the following components \(Fig. 4\):

- A ML surrogate **model**
- An **oracle** function to label unlabeled datapoints
- An **objective** function to select the new samples to be labeled (maximizing the uncertainty, maximizing some score, or other, more complex functions)

<img src="http://drive.google.com/uc?export=view&id=1FMDzm7pOt238ByYJwhiSHQkoqsA1EKqb" alt="drawing" width="500"/>

__Fig. 4:__ Generic active learning workflow



For this lab, we will construct a simple active learning loop to make the most out of a given budget of function calls to the (_somewhat_) expensive docking program _SMINA_.

In the previous parts, we constructed the *oracle* function that will label the molecules. To complete the *active learning workflow*, we need a *model* and an *objective* function.

- Model: We will use a simple Gaussian Process on ecfp fingerprints
- Objective function: We will maximize the uncertainty of the GP on the binding free energy prediction



In [ ]:
def get_random_idxs(df, n=10, seed=42):
    # Select molecules to create AL seed dataset
    np.random.seed(seed)
    return np.random.randint(0, len(df), n)


df = init_df_fields(df_mols)
df.head()

In [ ]:
from sklearn.gaussian_process.kernels import RBF

def train_gp(df) -> GaussianProcessRegressor:
    # retrieve for all labeled molecules the fingerprints and affinity labels
    X = np.vstack(df["fp"][df["sampled"]>=1].tolist())
    Y = np.vstack(df["true_affinity"][df["sampled"]>=1].tolist())
    # fit GP
    return GaussianProcessRegressor(kernel=RBF(length_scale=2.0,
                                               length_scale_bounds=(1e-1, 20.0)),
                                     random_state=0).fit(X,Y)

def predict_with_gp(df, gp):
    X = np.vstack(df["fp"].tolist())
    mean, std = gp.predict(X, return_std=True)
    df["pred_affinity"] = mean
    df["uncertainty"] = std
    return df

def samples_next(df, n: int = 10, sort_by_uncertainty = True) -> List[int]:
    original_df = df
    if sort_by_uncertainty:
        # largest uncertainty on top (aquisition function)
        ascending=False
        name="uncertainty"
    else:
        # best binders on top (most negative binding free energy)
        ascending=True
        name="pred_affinity"
    return df.sort_values(name, ascending=ascending)["idxs"].tolist()[:n]



In [ ]:
def get_results(output_dir, idxs):
    # retrieve binding affinity of optimal conformer for all labeled molecules
    values = []
    key = "minimizedAffinity"
    for idx in idxs:
        poses = dm.read_sdf(os.path.join(output_dir, f"poses_{idx}.sdf"),
                            as_df=True, mol_column="mols", n_jobs=-1,
                            sanitize=False)
        poses = poses.sort_values("minimizedAffinity",inplace=False)
        values.append(poses["minimizedAffinity"][0])
    return values

def format_df(df, affinities, sampled_idxs, iteration):
    # save label from oracle
    df["true_affinity"][sampled_idxs] = affinities
    df["sampled"][sampled_idxs] = iteration
    return df


In [ ]:
from copy import deepcopy

N_OF_AL_ITERATIONS = 5
FIRST_LOOP=True  # get random idxs at the first loop
N_OF_ORACLE_CALLS=3  # AL batch size
SELECT_BY_UNCERTAINTY=True
SEED = 42

docker=Docking("smina_inputs/receptor.pdbqt", box, num_poses=3)

ultimate_df = deepcopy(df)
for iteration in range(N_OF_AL_ITERATIONS):
    if FIRST_LOOP:
        FIRST_LOOP = False
        sampled_idxs= get_random_idxs(ultimate_df, n=N_OF_ORACLE_CALLS,
                                      seed=SEED).tolist()

    print(f"Selected idxs: {sampled_idxs}")

    # Create iteration directory
    output_dir = f"al_loop_{iteration}"
    os.makedirs(output_dir,exist_ok=True)

    # Select molecules to dock and dock them
    # (really slow on colab)
    mols_to_dock=ultimate_df["mols"].to_numpy()[sampled_idxs]
    docker.dock_multiple_mols(mols_to_dock, sampled_idxs, output_dir)

    # Get and store results
    affinities = get_results(output_dir, sampled_idxs)
    print(f"Obtained affinities in AL interation {iteration}: {affinities}")
    ultimate_df = format_df(ultimate_df, affinities, sampled_idxs, iteration+1)

    # re-train and use surrogate model
    GP = train_gp(ultimate_df)
    ultimate_df = predict_with_gp(ultimate_df, GP)
    sampled_idxs = samples_next(ultimate_df, N_OF_ORACLE_CALLS, SELECT_BY_UNCERTAINTY)




# Lab2 Recap

1. We have successfully run a small docked screening on the EGFR protein by preparing the protein and ligands, creating the binding box, running the docking and analyzing the results.

2. We have created a simple Gaussian Process model to predict the binding affinity of the ligands.

3. We have used Active Learning to optimize the docking surrogate model.

## Discussion & Homework

### Where to go next

Classical docking software relies on hand\-crafted, physics\-inspired functional forms as well as classical optimization routines to find docking poses and predict binding affinity. Several ML directions aim to improve on this:

- **DiffDock** trains a _diffusion model_ to predict the binding pose to _improve the accuracy of the pose_ and aims to _speed\-up_ the prediction by avoiding the optimization scheme.
- Several methods learn **ML scoring functions** using the PDBBind database to _improve the accuracy_ of the binding affinity prediction.
- **AlphaFold3** co\-folds the protein with the ligand within the binding site, reducing the need for high\-quality protein _holo crystal structures_, which are assumed to be known by Docking approaches.


### Limitations

* Docking programs can consider some _residue sidechains to be flexible_ during docking calculations to account for binding pocket flexibility. However, the _dynamic, adaptive nature_ of the protein\-ligand binding and the contribution of the conformational entropy of the protein is insufficiently considered by protein\-ligand docking. This can result in false positives: Even if the ligand finds a suitable pose in the binding pocket, _this position is not guaranteed_ until the protein is allowed to explore near\-minima conformations. Hence, short _molecular dynamics \(MD\) simulations_ are recommended to evaluate the stability of the predicted binding pose ([https://doi.org/10.2174/157340911795677602](https://doi.org/10.2174/157340911795677602)\).

* **Scoring functions** used by docking programs must be _cheap to compute_. While the accuracy is good enough to distinguish good poses from bad poses, it can have _problems sorting the best poses_. For example, while most popular docking programs are able to find the experimental pose in their calculations, this pose is rarely the best one of the proposed set. Furthermore, several retrospective studies have shown that _docking scores often poorly correlate with binding affinity_ (https://doi.org/10.1021/jm050362n), (https://doi.org/10.1039/c6cp01555g).  

* While blind docking is possible, in order to reduce computational cost, docking is often only performed on a subset of the protein \(typically around a known binding pocket\). _Choosing the correct binding site is a challenge_, if the binding pocket is not known a priori.

* To maximize the accuracy of the calculation, the ligand and protein structures must be _prepared appropriately_. Protonation states of amino acids and the ligands can be tricky to get right, especially in the case of \(potential\) tautomers. This introduces yet another cause to obtain inaccurate results.

Due to these limitations, Docking predictions tend to be not very accurate with a mean absolute error of 2.85 kcal/mol on PDBBind  [\(Vina\)](https://onlinelibrary.wiley.com/doi/10.1002/jcc.21334).

<img src="http://drive.google.com/uc?export=view&id=1qiOvjc7RtUtV8sNFBzmvm5Oa01xTzLom" alt="drawing" width="800"/>


#### Beyond Docking: Physically rigorous approaches using Molecular Dynamics

Binding Affinity is a _complex property_ that depends on the protein-ligand interaction, the solvation free energy, and
the entropy and enthalpy of the system. By only _considering a single frame_ of the optimally docked ligand conformation, the accuracy of Docking is inherently limited. In contrast, physically rigorous **free energy simulations** can model all of the phenomena outlined above. In particular, alchemical free energy methods such as free energy perturbation (**FEP**) have been highly successful in the last decade. To this end, **Molecular Dynamics** (MD) simulations are performed to sample the Boltzmann distribution of the protein-ligand complex, while continuously _turning off selection interactions_ between the protein and the ligand (much more complicated than that).

The accuracy of the MD simulation (and consequently the accuracy of the binding free energy estimate) critically depends on the _employed force field_. While to date, almost all free energy simulations are performed using classical force fields, **neural network potentials** (NNPs) promise increased accuracy by modeling higher body-order interactions. However, due to the significantly increased computational cost of NNPs compared to classical force fields, this increases the need for (ML-supported) **enhanced sampling** schemes to speed-up MD simulations.

Free energy simulations are much more expensive than Docking, increasing the demand for active learning approaches. Hence, **Active Learning FEP** is a promising research area.

#### Improving the active learning loop

The active learning loop and model we used for this workshop are quite simple and could be improved in several ways:

1. ##### **Binding Affinity Prediction:**
    
We used a simple _Gaussian Process (GP) with a RBF kernel trained on fingerprints_ to predict the binding affinity.
More sophisticated models could be used to improve the prediction accuracy.

Binding Affinity is a complex property that depends on the _3D structure_ (and dynamics) of the protein-ligand complex. However, our GP surrogate model is not even taking into consideration any 3D information.
Hence, using some _3D descriptors_ such as Smooth Overlap of Atomistic Position (SOAP) or Atomic Cluster Expansion (ACE) could improve the prediction.
Instead of these hand-crafted 3D feature extraction functions, learned representations from _graph neural networks_ could further improve the surrogate model.

2. ##### **Active Learning:**
    
We used a quite _simple aquisition function (max uncertainty)_. With this aquisition function, we are only interested in generating data such that our surrogate model improves fastest, i.e. we only care about _exploration_. However, if we are interested in finding suitable compounds as fast as possible, we can use more sophisticated acquisition functions to guide the optimization process in order to balance exploration and exploitation efforts.

Some of the more commons acquisition functions are:


- ##### _Probability of Improvement (PI)_

How likely is the objective function value improving over our current optimum?


<center><img src="http://drive.google.com/uc?export=view&id=10QDcY86Tn3tZRVZOQql5a45-KmssfDA-" alt="drawing" width="1200"/></center>

$$ PI(x) = \psi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big),$$

with 𝜇(𝑥)
 and 𝜎(𝑥) the mean and standard deviation of the regressor at 𝑥
, 𝑓
 is the objective function to be optimized with estimated maximum at 𝑥+
and 𝜉
 is a parameter controlling the exploration-exploitation tradeoff.  𝜓(𝑧)
 denotes the cumulative distribution function of a standard Gaussian distribution.

- ##### _Expected Improvement (EI)_

How much the onjective function can be expected to improve over our current optimum?

<center><img src="http://drive.google.com/uc?export=view&id=17rHWb1869QkLe0GNoctYhxroEP8CfK52" alt="drawing" width="1200"/></center>

$$ \begin{split}\begin{align*}
EI(x) = & (\mu(x) - f(x^+) - \xi) \psi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big) \\
& + \sigma(x) \phi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big),
\end{align*}\end{split} $$

Similar to the PI, but EI adds the probability density function 𝜙(𝑧) of a standard Gaussian distribution. EI takes into account the size of the improvement.


- ##### _Upper Confidence Bound (UCB)_

<center><img src="http://drive.google.com/uc?export=view&id=1D1M7_ymH8eM8NtBbejM1OIOcO0CBfVLJ" alt="drawing" width="1200"/></center>

$$ UCB(x) = \mu(x) + \beta \sigma(x), $$

 𝜇(𝑥)
 and 𝜎(𝑥)
 are the mean and standard deviation of the regressor and 𝛽
 is a parameter controlling the degree of exploration.

---

#### If you want to continue from here:

- As an exercise, you can try to _implement one of these acquisition functions_ and compare the results with the max uncertainty
approach while running a more in depth screening of the EGFR protein (from the day 1 Lab results).
A useful library to implement these acquisition functions is modAL: A modular active learning framework for Python3 (! pip install modAL-python).

- Can we build a more _sophisticated surrogate model_ such as a neural network to predict the binding affinity?

- From the 100 molecules from the screening of Lab1, can we screen down to 10 _good_ molecules?
